# Adsorption column breakthrough curve

Notebook to simulate the breakthrough curve and to fit the adsorbent parameters based on an experimental breakthrough curve

In [ ]:
import numpy as np

In [ ]:
# Function to calculate the ODE
def BT_ODE(x, t, n, D, v, dz, eps, k, qs, b, cin):
    # Temporary arrays to make the derivatives simpler
    gas_conc = np.zeros(n + 2)
    gas_conc[1:-1] = x[0::2]
    solid_conc = np.zeros(n + 2)
    solid_conc[1:-1] = x[1::2]
    
    # Ghost points
    gas_conc[0] = v *dz/D * (cin - x[0]) + x[2]
    gas_conc[n+1] = x[2 * n - 2]
    
    # Empty vector for the derivatives
    dxdt = np.zeros(2 * n)

    # Define derivatives
    for i in range(1, n+1):  
        dxdt[2*i-1] = k*(qs*b*gas_conc[i]/(1 + b*gas_conc[i]) - solid_conc[i])   # Adsorbed concentration
        diff = D * (gas_conc[i+1] - 2*gas_conc[i] + gas_conc[i-1])/(dz * dz)     # Diffusion term
        conv = v * (gas_conc[i] - gas_conc[i-1])/dz                              # Convection
        adso = (1-eps)/eps * dxdt[2*i-1]                                         # Adsorption
        dxdt[2*(i-1)] = diff - conv - adso                                       # Fluid conc
        
    return dxdt

In [ ]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Simulate the breakthrough curve
def BT_solve(x=[1.0, 15.0, 1.0], plot_flag=False):

    # Assign adsorbent parameters
    k = x[0]
    qs = x[1]
    b = x[2]

    # Read experimental data
    exp_data = np.loadtxt("../data/BREAKLDF25050.dat", skiprows=1)

    # Parameters
    n = 10      # Number of grid points
    D = 0.00001 # Diffusion coefficient [m^2/s]
    v = 0.5     # Intersticial velocity [m/s]
    eps = 0.3   # Bed void fraction
    cin = 1     # Input concentration [mol/m^3]
    dz = 1/(n-1.0) # Grid spacing

    # Find the first time past 100s
    result = np.where(exp_data[:,0] >= 100)
    idx = result[0][0] + 1
    c_0 = np.zeros(2*n)       # Initial fluid and solid concentration [mol/m^3]
    t = exp_data[0:idx, 0]          # Set the time spacing for the integration

    sol = odeint(BT_ODE, c_0, t, args=(n, D, v, dz, eps, k, qs, b, cin))

    if plot_flag:
        plt.plot(t, sol[:, 2*n-2], 'b', label='Simulation')
        plt.plot(t, exp_data[0:idx, 1], 'g--', label='Experiment')
        plt.legend(loc='best')
        plt.xlabel('Time')
        plt.ylabel("Fluid concentration")
        plt.grid()
        plt.show()
        
    # Calculate the squared error between the experiment and simulation
    error = np.sum((sol[:, 2*n-2] - exp_data[0:idx, 1])**2)
    
    return error

In [ ]:
# Optimise the material properties
from scipy.optimize import minimize

x0 = np.array([1.0, 15.0, 1.0])
res = minimize(BT_solve, x0, method='nelder-mead', args=(False),
               options={'xatol': 1e-8, 'maxiter': 20, 'disp': True});

# Plot the result
BT_solve(res.x, plot_flag=True)